In [ ]:
!pip install -q -U google-genai
import duckdb
import pandas as pd
import os
from google import genai
from dotenv import load_dotenv
import os
import json
import traceback


os.chdir("../../../")
%load_ext sql
conn = duckdb.connect("duckdb/main.duckdb")
write_conn = duckdb.connect("duckdb/main.duckdb")  # For UPDATEs and COPY
%sql conn --alias duckdb


#%config SqlMagic.displaylimit = None

In [ ]:
%%sql
/*SELECT table_name
FROM information_schema.tables
WHERE table_type = 'BASE TABLE'*/

show table transkribus_metadata

In [ ]:
%%sql
select sum(DB_), avg(DB_) FROM(
select newspaper, count(distinct block)  AS DB_  from full_text where 1=1 and len(text) > 1 and block <> 'All' GROUP BY 1
) a

In [ ]:
conn.close()
write_conn.close()

In [ ]:
%%sql
SELECT PV.Text
FROM Full_Text PV
LEFT JOIN Full_Text NV
ON NV.Newspaper = PV.Newspaper
AND NV.Source = PV.Source
AND NV.Type = PV.Type
AND NV.Block = PV.Block
AND NV.Source = 'pro'
WHERE NV.Text IS NULL
AND PV.Source = 'split_blocks.py'
AND PV.Type = 'N/A'
AND len(PV.Text) > 50
AND PV.Newspaper = 'eestikirikeelk19260225.1.6'
AND PV.Block = 'TB00001'


In [ ]:
conn.close()
write_conn.close()

In [ ]:
# Load environment variables
load_dotenv()

# Retrieve API key
GEMINI_API_KEY = os.getenv("GEM")
if not GEMINI_API_KEY:
    raise ValueError("Environment variable 'GEM' is not set or invalid.")

# Initialize the client
client = genai.Client(api_key=GEMINI_API_KEY)
tbu_model = "gemini-2.5-pro"
prompt_tmp = 'original_olmocr'
block = 'All'

prompt_template = conn.execute(
    f"""
/
SELECT Prompt.Text
FROM Prompt
WHERE Text IS NOT NULL
AND Version = 1 -- Change if there are multiple version, should most likely be changed to take in max value
AND Prompt.Type = {prompt_tmp}
    """
)

cursor = conn.execute(
    f"""
/* Check for newspapers that do not have text from current model */
SELECT State.Newspaper, Full_Text.Text
FROM State
LEFT JOIN Full_Text
ON State.Newspaper = Full_Text.Newspaper
AND Full_Text.Source = '{tbu_model}'
AND Full_Text.Type = '{prompt_template}'
AND Full_Text.Block = {block}
WHERE Full_Text.Text IS NULL

    """
)



row = cursor.fetchone()
results = []

while row :
    try: 
        newspaper_id, existing_text = row
    
        if existing_text is not None:
                row = cursor.fetchone()
                continue
        # Verify image path
        image_path = f"/mnt/3de36453-6164-4568-91b5-ae973509273e/Git/EE-Gothic-Script-OCR/src/datasets/silver/scanned_jpg/{newspaper_id}.jpg"
        if not os.path.exists(image_path):
            print(f"Image not found, skipping: {image_path}")
            row = cursor.fetchone()
            continue

        
        json_path = f"/mnt/3de36453-6164-4568-91b5-ae973509273e/Git/EE-Gothic-Script-OCR/src/datasets/silver/scanned/{newspaper_id}/json/fulltext.json"
        if not os.path.exists(json_path):
            print(f"JSON not found, skipping: {json_path}")
            row = cursor.fetchone()
            continue
        # Load JSON layout and convert to indented string
        with open(json_path, "r", encoding="utf-8") as f:
            layout_json = json.load(f)
        layout_text = json.dumps(layout_json, indent=2, ensure_ascii=False)
        # Upload file
        if layout_text and os.path.exists(image_path):

            my_file = client.files.upload(file=image_path)

            # Prompt
            prompt = [
                image_path,
                {
                    "text": f"""Below is the image of one page of a PDF document , as well as some raw textual content that
                        was previously extracted for it that includes position information for each image and
                        block of text ( The origin [0 x0 ] of the coordinates is in the upper left corner of the
                        image ).
                        Just return the plain text representation of this document as if you were reading it
                        naturally .
                        Turn equations into a LaTeX representation , and tables into markdown format . Remove the
                        headers and footers , but keep references and footnotes .
                        Read any natural handwriting .
                        This is likely one page out of several in the document , so be sure to preserve any sentences
                        that come from the previous page , or continue onto the next page , exactly as they are .
                        If there is no text at all that you think you should read , you can output null .
                        Do not hallucinate .
                        RAW_TEXT_START
                        { layout_text }
                        RAW_TEXT_END
                """
                        }
                    ]

            # Generate content
            response = client.models.generate_content(
                model=tbu_model,
                contents=prompt,
            )
            response_text = response.text.strip() if response and response.text else None
            # Collect result
            df = pd.DataFrame([{
                "Newspaper": newspaper_id,
                "Source": tbu_model,
                "Text": response_text
                }])
            write_conn.register("df", df)

            write_conn.execute("""
            INSERT INTO Full_Text (Text, Source, Type, Newspaper, Timestamp)
            SELECT df.Text, df.Source, {prompt_tmp}, df.Newspaper, CURRENT_TIMESTAMP
            FROM df
            """)

    except Exception as e:
        print(f"[ERROR] While processing {row[0]}: {e}")
        traceback.print_exc()

    


    row = cursor.fetchone()  


# Convert results to DataFrame
write_conn.execute("""
        COPY Full_Text TO 'duckdb/full_text.parquet' (FORMAT PARQUET);
        """)


In [ ]:

# Convert results to DataFrame
write_conn.execute("""
        COPY Full_Text TO 'duckdb/full_text.parquet' (FORMAT PARQUET);
        """)


In [ ]:
%%sql
select * 
FROM Full_Text
where
1=1
and source  IN( 'gemini-2.5-pro','dea.digar.coord')
--and newspaper = 'eestikirikeelk19260204.1.4' 
--and block ='TB00005'
and timestamp > '2025-08-03 15:00:24.906000'
AND timestamp < '2025-08-03 16:00:24.906000'
order by newspaper,block
--GROUP BY Newspaper,Block
--having Train_input is not null and train_output is not null


In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS OLMOCR_train  AS(
SELECT 
Newspaper,
Block,
MAX(CASE WHEN Source = 'dea.digar.coord' then trim(text) else null END)  AS Train_Input,
MAX(CASE WHEN Source = 'gemini-2.5-pro' then trim(text) else null END)  AS Train_Output
FROM Full_Text
where source  IN( 'gemini-2.5-pro','dea.digar.coord')
GROUP BY Newspaper,Block
having Train_input is not null and train_output is not null
) WITH DATA 

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS OLMOCR_train_nc  AS(
SELECT 
Newspaper,
Block,
MAX(CASE WHEN Source = 'dea.digar' then trim(text) else null END)  AS Train_Input,
MAX(CASE WHEN Source = 'gemini-2.5-pro' then trim(text) else null END)  AS Train_Output
FROM Full_Text
where source  IN( 'gemini-2.5-pro','dea.digar')
GROUP BY Newspaper,Block
having Train_input is not null and train_output is not null
) WITH DATA 

In [ ]:
write_conn.execute("""
        COPY olmocr_train_nc TO 'src/datasets/gold/olmocr_train_nc.parquet' (FORMAT PARQUET);
        """)

## Fill original text + coordinates

In [ ]:
rows = conn.execute(
    """
    SELECT State.Newspaper
    FROM State
    LEFT JOIN Full_Text
    ON State.Newspaper = Full_Text.Newspaper
    AND Full_Text.Source = 'dea.digar.coord'
    AND Full_Text.Type = 'N/A'
    WHERE Full_Text.Text IS NULL
    """
).fetchall()

for row in rows:
    try:
        newspaper_id = row[0]
        
        image_path = f"src/datasets/silver/scanned_jpg/{newspaper_id}.jpg"
        if not os.path.exists(image_path):
            print(f"Image not found, skipping: {image_path}")
            continue
        
        json_path = f"src/datasets/silver/scanned/{newspaper_id}/json/fulltext.json"
        if not os.path.exists(json_path):
            print(f"JSON not found, skipping: {json_path}")
            continue
        
        elif os.path.exists(image_path):  # Only proceed if image exists
            with open(json_path, "r", encoding="utf-8") as f:
                layout_json = json.load(f)
            layout_json_string = json.dumps(layout_json, ensure_ascii=False)


            df = pd.DataFrame([{
                "Newspaper": newspaper_id,
                "Source": 'dea.digar.coord',
                "Text": layout_json_string,
                "Type": 'N/A'
            }])

            write_conn.register("df", df)
            write_conn.execute("""
                INSERT INTO Full_Text (Text, Source, Type, Newspaper, Timestamp)
                SELECT df.Text, 'dea.digar.coord', 'N/A', df.Newspaper, CURRENT_TIMESTAMP
                FROM df
                WHERE NOT EXISTS (
                    SELECT 1 FROM Full_Text IQ
                    WHERE IQ.Source = df.Source
                    AND IQ.Type = df.Type
                    AND IQ.Newspaper = df.Newspaper
                )
            """)
    
    except Exception as e:
        print(f"[ERROR] While processing {row[0]}: {e}")
        traceback.print_exc()



In [ ]:
%%sql
SELECT * FROM Full_Text WHERE 1=1  and text like'%Joobnu tulistas%'


In [ ]:
%%sql
/* Check for newspapers that do not have text from current model */
SELECT State.Newspaper, Full_Text.Text, Full_Text.Timestamp
FROM State
LEFT JOIN Full_Text
ON State.Newspaper = Full_Text.Newspaper
AND Full_Text.Type = 'gemini-2.5-flash'
WHERE Full_Text.Text IS NULL


## Blocks

In [ ]:
%%sql
SELECT 
Newspaper,
Block,
Train_Input,
--case when pro is not null then 'Pro' Else 'FLASH' END AS Ind,
--COALESCE(Pro,FLash) AS Train_Output
Split_blocks_No_Coords as Train_Output

FROM(
SELECT 
Newspaper,
Block,
MAX(CASE WHEN Source = 'split_blocks_pypdf' then trim(text) else null END)  AS Train_Input,
MAX(CASE WHEN Source IN( 'split_blocks_no_coords') then trim(text) else null END)  AS Split_blocks_No_Coords
FROM Full_Text
where source  IN( 'split_blocks_pypdf','split_blocks_no_coords')
AND Block <> 'All'
AND NOT EXISTS(
    SELECT 1 FROM OLMOCR_TRAIN 
    WHERE OLMOCR_TRAIN.Newspaper = Full_text.newspaper
    AND OLMOCR_TRAIN.BLock = FUll_Text.newspaper
)
AND EXISTS(
SELECT 1 FROM BLOCK_QUALITY BQ
WHERE BQ.NEwspaper = Full_Text.newspaper
AND left(BQ.block,7) = full_text.block
AND Bq.Correct_Ind = 'Y' 
)
AND Full_text.newspaper = 'kaja19290813-1.1.8'
GROUP BY Newspaper,Block
having Train_input is not null and Split_blocks_No_Coords is not null) a
Where len(Train_Output) > 0

In [ ]:
%%sql

select *  from full_text 
where block <> 'All' 
--and source = 'gpt-4o'
and newspaper = 'kaja19270710-1.1.7'
and block = 'TB00004'
 --and newspaper = 'jarvateatajaew19270610.1.3'
  --and trim(block)= 'TB00037'

In [ ]:
%%sql
/*
select
sum(case when Pro is null and GPT is not null then 1 else 0 end ) GPT_ONLY,
sum(case when Pro is not  null and GPT is  null then 1 else 0 end ) PRO_ONLY,
sum(case when Pro is not  null and GPT is  not null then 1 else 0 end) BOTH_
FROM(*/

SELECT 
Newspaper,
Block,
MAX(CASE WHEN Source IN( 'split_blocks_no_coords' )then trim(text) else null END)  AS Manual,
MAX(CASE WHEN Source IN( 'gemini-2.5-pro') then trim(text) else null END)  AS Pro,
MAX(CASE WHEN Source IN( 'gpt-4o') then trim(text) else null END)  AS GPT
FROM Full_Text
where source  IN( 'gemini-2.5-pro','gpt-4o','split_blocks_no_coords')
AND Block <> 'All'
AND  EXISTS(
SELECT 1 FROM BLOCK_QUALITY BQ
WHERE BQ.NEwspaper = Full_Text.newspaper
AND left(BQ.block,7) = full_text.block
AND Bq.Correct_Ind = 'Y' 
)
GROUP BY Newspaper,Block
having gpt is  not null and  pro is not null 



--) A


In [ ]:
%%sql
select count(*) FROM OLMOCR_TRAIN 

In [ ]:
%%sql
INSERT INTO OLMOCR_TRAIN (Newspaper, Block,Train_Input, Train_Output)
SELECT 
Newspaper,
Block,
Train_Input,
--case when pro is not null then 'Pro' Else 'FLASH' END AS Ind,
--COALESCE(Pro,FLash) AS Train_Output
Pro as Train_Output
FROM(
SELECT 
Newspaper,
Block,
MAX(CASE WHEN Source = 'split_blocks.py' then trim(text) else null END)  AS Train_Input,
MAX(CASE WHEN Source IN( 'gemini-2.5-pro') then trim(text) else null END)  AS Pro
--MAX(CASE WHEN Source IN( 'gemini-2.5-flash') then trim(text) else null END)  AS Flash
FROM Full_Text
where source  IN( 'gemini-2.5-pro','gemini-2.5-flash','split_blocks.py')
AND Block <> 'All'
AND NOT EXISTS(
    SELECT 1 FROM OLMOCR_TRAIN 
    WHERE OLMOCR_TRAIN.Newspaper = Full_text.newspaper
    AND OLMOCR_TRAIN.BLock = FUll_Text.BLock
)
AND NOT EXISTS(
SELECT 1 FROM BLOCK_QUALITY BQ
WHERE BQ.NEwspaper = Full_Text.newspaper
AND left(BQ.block,7) = full_text.block
AND Bq.Correct_Ind = 'Y' 
)
GROUP BY Newspaper,Block
having Train_input is not null and PRO is not null
) SQ  WHERE len(Train_Output) > 0 and len(Train_input) > 0

UNION 

SELECT 
Newspaper,
Block,
Train_Input,
--case when pro is not null then 'Pro' Else 'FLASH' END AS Ind,
--COALESCE(Pro,FLash) AS Train_Output
Split_blocks_No_Coords as Train_Output
FROM(
SELECT 
Newspaper,
Block,
MAX(CASE WHEN Source = 'split_blocks_pypdf' then trim(text) else null END)  AS Train_Input,
MAX(CASE WHEN Source IN( 'split_blocks_no_coords') then trim(text) else null END)  AS Split_blocks_No_Coords
FROM Full_Text
where source  IN( 'split_blocks_pypdf','split_blocks_no_coords')
AND Block <> 'All'
AND NOT EXISTS(
    SELECT 1 FROM OLMOCR_TRAIN 
    WHERE OLMOCR_TRAIN.Newspaper = Full_text.newspaper
    AND OLMOCR_TRAIN.BLock = FUll_Text.BLock
)
AND EXISTS(
SELECT 1 FROM BLOCK_QUALITY BQ
WHERE BQ.NEwspaper = Full_Text.newspaper
AND left(BQ.block,7) = full_text.block
AND Bq.Correct_Ind = 'Y' 
)
GROUP BY Newspaper,Block
having Train_input is not null and Split_blocks_No_Coords is not null
) SQ  WHERE len(Train_Output) > 0 and len(Train_input) > 0


In [ ]:
write_conn.execute("""
        COPY olmocr_train TO 'src/datasets/gold/olmocr_train.parquet' (FORMAT PARQUET);
        """)

## Prompt modification

In [ ]:
%%sql
CREATE TABLE Prompt ( Type VARCHAR, Version SMALLINT, Text VARCHAR);



In [ ]:
prompt_template = """
Below is the image of a block from a PDF image, as well as some raw textual content that
was previously extracted for it that includes position information for each row(The origin [0 x0] of the coordinates is in the upper left corner of the
image).
Just return the plain text representation of this document as if you were reading it
naturally.
This is likely one article out of several in the document, so be sure to preserve any sentences
that come from the previous page, or continue onto the next page, exactly as they are.
If there is no text at all that you think you should read, you can output null.
Do not hallucinate.
RAW_TEXT_START
{layout_text}
RAW_TEXT_END
"""

conn.execute(
        """
        INSERT INTO Prompt (Type, Version, Text)
        VALUES (?, ?, ?)
        """,
        ('block_prompt', 1, prompt_template)
    )

In [ ]:
write_conn.execute("""
        COPY Prompt TO 'duckdb/prompt.parquet' (FORMAT PARQUET);
        """)

### Convert full page results 

In [ ]:
%%sql
SELECT
PV.Newspaper,
--PV.Block,
string_agg('{'|| PV.Block ||', ' || PV.Text||'}', '' ORDER  BY PV.Block) AS Combined_Text
,FV.Text
FROM Full_Text PV
LEFT JOIN Full_Text NV
ON NV.Newspaper = PV.Newspaper
AND NV.Source = PV.Source
AND NV.Type = PV.Type
AND NV.Block = PV.Block
AND NV.Source = 'pro'
JOIN FUll_Text FV
ON FV.Newspaper = PV.Newspaper
--ND FV.Type like '%originial%'
AND FV.Source = 'gemini-2.5-pro'
AND FV.BLock = 'All'
LEFT JOIN block_quality
ON block_quality.Newspaper  = PV.Newspaper
AND left(block_quality.Block,7) = PV.Block
AND block_quality.Correct_Ind = 'Y'
WHERE NV.Text IS NULL
AND BLock_Quality.Newspaper is null
AND PV.Source = 'split_blocks.py'
AND PV.Type = 'N/A'
AND len(PV.Text) > 300
AND PV.Newspaper = 'kaja19350701-1.1.3'
--AND PV.Block = 'TB00001'
GROUP BY PV.Newspaper
--, PV.Block
, FV.Text
limit 1


In [ ]:
# Load environment variables
load_dotenv()

# Retrieve API key
GEMINI_API_KEY = os.getenv("GEM")
if not GEMINI_API_KEY:
    raise ValueError("Environment variable 'GEM' is not set or invalid.")

# Initialize the client
client = genai.Client(api_key=GEMINI_API_KEY)
tbu_model = "gemini-2.5-flash"
prompt_tmp = 'original_olmocr'
block = 'pagetorow'


cursor = conn.execute(
    f"""
/* Check for newspapers that do not have text from current model */
SELECT
PV.Newspaper,
--PV.Block,
string_agg('{'|| PV.Block ||', ' || PV.Text||'}', '' ORDER  BY PV.Block) AS Combined_Text
,FV.Text
FROM Full_Text PV
LEFT JOIN Full_Text NV
ON NV.Newspaper = PV.Newspaper
AND NV.Source = PV.Source
AND NV.Type = PV.Type
AND NV.Block = PV.Block
AND NV.Source = 'pro'
JOIN FUll_Text FV
ON FV.Newspaper = PV.Newspaper
--ND FV.Type like '%originial%'
AND FV.Source = 'gemini-2.5-pro'
AND FV.BLock = 'All'
LEFT JOIN block_quality
ON block_quality.Newspaper  = PV.Newspaper
AND block_quality.Block = PV.Block
AND block_quality.Correct_Ind = 'Y'
WHERE NV.Text IS NULL
AND BLock_Quality.Newspaper is null
AND PV.Source = 'split_blocks.py'
AND PV.Type = 'N/A'
AND len(PV.Text) > 300
--AND PV.Newspaper = 'eestikirikeelk19260225.1.6'
--AND PV.Block = 'TB00001'
GROUP BY PV.Newspaper
--, PV.Block
, FV.Text
    """
)



row = cursor.fetchone()
results = []

while row :
    try: 
        newspaper_id, text_with_coords, fixed_text = row
    

            # Prompt
        prompt = [
    {
        "text": (
            "You are provided with two texts:\n\n"
            "1) The original OCR text with block IDs and coordinates:\n"
            f"{text_with_coords}\n\n"
            "2) The corrected text without coordinates:\n"
            f"{fixed_text}\n\n"
            "Please compare these texts and try to figure out where the corrected lines are in the original version.\n"
            "Output Block_Id, corrected_text , coordinates for each line"
        )
    }
]


            # Generate content
        response = client.models.generate_content(
                model=tbu_model,
                contents=prompt,
            )
        response_text = response.text.strip() if response and response.text else None
        # Collect result
        df = pd.DataFrame([{
            "Newspaper": newspaper_id,
            "Source": tbu_model,
            "Text": response_text
            }])
        write_conn.register("df", df)
        write_conn.execute("""
        INSERT INTO Full_Text (Text, Source, Type, Newspaper, Timestamp)
        SELECT df.Text, df.Source, 'pagetorow', df.Newspaper, CURRENT_TIMESTAMP
        FROM df
        """)

    except Exception as e:
        print(f"[ERROR] While processing {row[0]}: {e}")
        traceback.print_exc()

    print(response_text)
    break

    row = cursor.fetchone()  


# Convert results to DataFrame
#write_conn.execute("""
#        COPY Full_Text TO 'duckdb/full_text.parquet' (FORMAT PARQUET);
#        """)
